# 实验一、单井稳定流求参

## 影响半径经验公式

吉哈尔特（承压水）：

$$
R =10 s_w \sqrt{K}
$$

库萨金（潜水）：

$$
R =2 s_w \sqrt{KH_0}
$$

式中，$K$ — 渗透系数，$m/d$；$s$ — 设计降深，$m$；$H_0$  — 自底板算起的含水层静止水位（厚度），$m$。

单孔抽水时可构造迭代公式计算 $K$ 及 $R$：

承压井：

$$
\left\{
  \begin{array}{rl}
  K&=\frac{Q}{2\pi Ms_w}\ln\frac{R}{r_w} \\
  R&=10s_w\sqrt{K}\\
  \end{array}
 \right.
$$

潜水井：

$$
\left\{\begin{array}{rl}
K&=\frac{Q}{\pi (2H_0-s_w)s_w}\ln\frac{R}{r_w}\\
R&=2s_w\sqrt{KH_0}\\
\end{array}\right.
$$

## 程序设计

推荐使用 NumPy 库。NumPy 是一个开放源代码的 Python 库，在科学和工程学的每个领域中都有应用，为 Python 的通用标准库。 NumPy API 已在 Pandas，SciPy，Matplotlib，scikit-learn，scikit-image 等软件包中广泛使用。NumPy 库包含多维数组和矩阵数据结构，其中的 ndarray 为齐次 n 维数组对象。NumPy可以对数组和矩阵进行高效计算，并且提供了庞大的高级数学函数库。

NumPy 进行数组运算非常有效，其中的 numpy.vectorize 将自定义的函数向量化，可以接受向量参数，并以向量返回结果，处理向量非常方便；numpy.array 将 Python 列表转化为 NumPy 数组；numpy.ones、numpy.zeros 用于构造元素为 1 或 0 的数组。常用的数学函数在 NumPy 中都有对应的形式，如 numpy.sin 、numpy.abs 、numpy.exp 等。

NumPy 库使用前需要导入：

In [1]:
import numpy as np
# 设置浮点数显示方式
np.set_printoptions(precision=4)

以承压水影响半径的经验公式为例，定义一个子程序 empirical，通过循环迭代，当两次计算结果差值绝对值小于误差限时终止计算。

In [2]:
def empirical(rw, M, Q, sw):
    K0 = 10
    R0 = 10*sw*np.sqrt(K0)
    while True:
        R = 10*sw*np.sqrt(K0)
        K = 0.5*Q*np.log(R/rw)/M/sw/np.pi
        if np.abs(R - R0) < 1.0e-8 and np.abs(K - K0) < 1.0e-6:
            break
        else:
            K0 = K
            R0 = R
    return K, R

用一次抽水试验数据计算：

In [3]:
rw = 0.4    # 井半径
M = 16.5    # 含水层厚度
Q = 320.54  # 抽水量
s = 1.16    # 降深值

K, R =empirical(rw, M, Q, s)

print('  K = {:.4f} m^2/d'.format(K))
print('  R = {:.4f} m'.format(R))

  K = 12.3221 m^2/d
  R = 40.7192 m


计算多次抽水试验数据，向量化函数非常方便：

In [4]:
vempirical= np.vectorize(empirical) # 将 empirical 函数向量化

rw = np.ones(3)*0.4  # 井半径向量
M = np.ones(3)*16.5  # 含水层厚度向量
Q = np.array([320.54, 421.63, 536.54])  # 抽水量向量
s = np.array([1.16, 1.60, 1.90])        # 降深值向量

K, R = vempirical(rw, M, Q, s)

print('  K = {} m^2/d'.format(K))
print('  R = {} m'.format(R))

  K = [12.3221 12.5963 14.1221] m^2/d
  R = [40.7192 56.786  71.4009] m
